In [221]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import pycountry
import seaborn as sns

In [238]:
pd.options.mode.chained_assignment = None  # default='warn'

## 1. Splite dataset into three part

In [381]:
train_file = pd.read_csv('train_advance.csv')

In [68]:
test_file = pd.read_csv('test_advance.csv.zip')

In [382]:
train_file

,Unnamed: 0.1,Unnamed: 0,location_id,profile_id,sid,sid_profile,post_id,post_type,description,numbr_likes,...,region,cd,phone,aj_exact_city_match,aj_exact_country_match,blurb,dir_city_name,dir_country_name,lat,lng
0,11420,11755,340283947,829324748,28919418,295070,Bka_6xunYma,1,Cheers to wine O‚Äôclock ü•Ç (üì∏: @aob1085),4985,...,NaN,ES,+34 936 65 77 41,False,False,NaN,El Prat de Llobregat,Spain,2,41
1,24264,24967,2631322,199074048,42114489,4342099,BxaZS4fA_nV,1,#malcapuyaisland #bananaisland #bulogisland,57,...,NaN,PH,NaN,True,False,NaN,Coron,Philippines,120,11
2,16286,16768,668571209,3280121227,19193237,2554203,BxmEzQUFocM,1,üî∏\\nSchwarzkehlchen (Saxicola rubicola) ‚Ä¢...,1576,...,Saxony-Anhalt,DE,NaN,False,True,NaN,NaN,NaN,11,51
3,11139,11467,238461670,5625643374,11733129,3465973,Bjq5KFGlJvM,1,Eared Grebe spotted at one of my favourite spo...,29,...,NaN,CA,NaN,True,False,NaN,NaN,NaN,-118,52
4,27320,28111,1005642734,1557071010,8540244,3541402,BweviowATmj,1,–ü–õ–ê–ö–ê–¢–¨ –í –ú–û–°–¢–ê–• - –≠–¢–û –ù–û–†...,1487,...,NaN,RU,79214201362,False,False,—Ü–µ–Ω—Ç—Ä –∏–Ω–∏—Ü–∏–∞—Ü–∏–∏ —Ç–≤–æ—Ä—á–µ—Å—Ç...,Aptekarskiy,Russia,30,59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22919,15908,16376,647709719023665,291380675,35417574,4486612,8qrm6vipMw,1,"E per chi,come noi ha paura dell'aereo...PENSA...",11056,...,NaN,NaN,390274852200,False,False,NaN,Vizzola Ticino,Italy,8,45
22920,15415,15870,216083829,7105966283,8355365,3196327,BvAIZlSldH2,1,"üå≥üçÉ Some people want material things. Me,...",1050,...,NaN,PR,17873223554,False,False,NaN,Las Mercedes,Puerto Rico,-66,18
22921,4428,4554,269593276871624,412311921,34484647,4318270,Bb_MBdsFXH_,2,Heading to the Desa dairy farm for a quick fil...,2581,...,NaN,MY,NaN,False,False,NaN,NaN,NaN,116,5
22922,16086,16562,216841346,1597259427,4306777,888760,BpNHCyugy7f,1,Ran my first unofficial 8K Race. #lululemongho...,77,...,NaN,US,NaN,False,False,LIVE webcam from Oak Street Beach in Chicago.,NaN,NaN,-87,41


In [383]:
train_file.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'location_id', 'profile_id', 'sid',
       'sid_profile', 'post_id', 'post_type', 'description', 'numbr_likes',
       'number_comments', 'profile_name', 'firstname_lastname', 'following',
       'followers', 'n_posts', 'url', 'is_business_account', 'name', 'zip',
       'city', 'region', 'cd', 'phone', 'aj_exact_city_match',
       'aj_exact_country_match', 'blurb', 'dir_city_name', 'dir_country_name',
       'lat', 'lng'],
      dtype='object')

In [384]:
Locations_file = train_file[['sid','name','zip','city','region','cd','phone','aj_exact_city_match',
                           'aj_exact_country_match','blurb','dir_city_name','dir_country_name','lat',
                           'lng']]

### 1.1 Location_file

In [385]:
Locations_file.isnull().sum()

sid                           0
name                          0
zip                        9966
city                       1907
region                    22529
cd                         1231
phone                     12956
aj_exact_city_match         335
aj_exact_country_match      335
blurb                     16073
dir_city_name             12263
dir_country_name          12263
lat                           0
lng                           0
dtype: int64

### 1.1.1 lat , lng

we can find the lat and lng are opposite, we need fix them. After we fix them, we use the google map to test some points. Most of points are correct, Which means the **dir_city_name** , **dir_country_name** can be deleted 

In [67]:
fig = px.scatter_geo(Locations_file, lat = 'lng', lon = 'lat')
fig.show()

###   1.1.2 city, region, cd, ZIP, Name

We can find when cd is null, are the other columns are null too, just **delete all null value accrordng to the cd**. From the google, we can get the **cd** can represent the **zip** , **city**, **region** and and **name** .

In [386]:
temp = Locations_file[Locations_file['cd'].isna()]
temp = temp[['zip','cd','city','region']]

In [387]:
temp[pd.isnull(temp).any(axis=1)]

,zip,cd,city,region
6,NaN,NaN,NaN,NaN
63,NaN,NaN,NaN,NaN
65,141075,NaN,NaN,NaN
71,NaN,NaN,NaN,NaN
86,NaN,NaN,NaN,NaN
...,...,...,...,...
22841,NaN,NaN,NaN,NaN
22855,NaN,NaN,NaN,NaN
22871,NaN,NaN,NaN,NaN
22878,NaN,NaN,NaN,NaN


Let we check cd it correct or not, here we sue the pycountry to get all the country code and we can find the country code are the correct

In [388]:
cd_check = Locations_file[~Locations_file['cd'].isna()]

In [389]:
a = []
for i in pycountry.countries:
    a.append(i.alpha_2)

In [390]:
for i in cd_check['cd']:
    if i not in a:
        print(i)

### 1.1. 3 check the aj_exact_city_match and  aj_exact_country_match

From the 1.1, we can find there are some missing values in aj_exact_city_match and  aj_exact_country_match. Let we check it there any other combinations.

In [391]:
Locations_file[['aj_exact_city_match','aj_exact_country_match']].value_counts()

aj_exact_city_match  aj_exact_country_match
False                False                     15626
True                 False                      6568
False                True                        395
dtype: int64

Check the null value, and we can find when the aj_exact_city_match,aj_exact_country_match are null, **the cd must none**.

In [392]:
temp = Locations_file[['aj_exact_city_match','aj_exact_country_match']]
Locations_file[['aj_exact_city_match','aj_exact_country_match','cd']].iloc[temp[pd.isnull(temp).any(axis=1)].index]


,aj_exact_city_match,aj_exact_country_match,cd
102,NaN,NaN,NaN
228,NaN,NaN,NaN
362,NaN,NaN,NaN
367,NaN,NaN,NaN
447,NaN,NaN,NaN
...,...,...,...
22743,NaN,NaN,NaN
22790,NaN,NaN,NaN
22793,NaN,NaN,NaN
22795,NaN,NaN,NaN


In [393]:
c = Locations_file[['aj_exact_city_match','aj_exact_country_match','cd']].iloc[temp[pd.isnull(temp).any(axis=1)].index]
c.describe()

,aj_exact_city_match,aj_exact_country_match,cd
count,0,0,0
unique,0,0,0
top,NaN,NaN,NaN
freq,NaN,NaN,NaN


According to the Locations_file, we can find:
 1. the lat and lon are oposite.
 2. cd have no abivious errors, and we can use **cd, lat and lon** to repesent  **city, region,ZIP, Name**
 3. When aj_exact_city_match and aj_exact_country_match are null, the cd must be null
 4. we can set **phone** into true or false to judge wether there are have phone number
 
So, we should change the lat and lon, delete the null value in cd direcly and drop city, region,ZIP, Name directly

In [394]:
# change name
train_file = train_file.rename(columns={'lat': 'lng', 'lng': 'lat'}) 

# delete null value in cd
cd_delete = train_file[train_file['cd'].isnull()].index.tolist()
train_file = train_file.drop(cd_delete).reset_index(drop=True)

# change phone into true of false
train_file['phone'] = np.where(train_file['phone'].isnull(), False, True)

# delete city, region,ZIP, Name 
train_file = train_file.drop(columns=['zip','city','region','name','blurb','dir_city_name','dir_country_name'])

In [395]:
train_file.shape

(21693, 24)

In [396]:
train_file.head(5)

,Unnamed: 0.1,Unnamed: 0,location_id,profile_id,sid,sid_profile,post_id,post_type,description,numbr_likes,...,followers,n_posts,url,is_business_account,cd,phone,aj_exact_city_match,aj_exact_country_match,lng,lat
0,11420,11755,340283947,829324748,28919418,295070,Bka_6xunYma,1,Cheers to wine O‚Äôclock ü•Ç (üì∏: @aob1085),4985,...,594895,3612.0,likeshop.me/oprahmagazine,True,ES,True,False,False,2,41
1,24264,24967,2631322,199074048,42114489,4342099,BxaZS4fA_nV,1,#malcapuyaisland #bananaisland #bulogisland,57,...,428,95.0,NaN,False,PH,False,True,False,120,11
2,16286,16768,668571209,3280121227,19193237,2554203,BxmEzQUFocM,1,üî∏\\nSchwarzkehlchen (Saxicola rubicola) ‚Ä¢...,1576,...,36618,1259.0,i111i.myportfolio.com,True,DE,False,False,True,11,51
3,11139,11467,238461670,5625643374,11733129,3465973,Bjq5KFGlJvM,1,Eared Grebe spotted at one of my favourite spo...,29,...,126,39.0,NaN,False,CA,False,True,False,-118,52
4,27320,28111,1005642734,1557071010,8540244,3541402,BweviowATmj,1,–ü–õ–ê–ö–ê–¢–¨ –í –ú–û–°–¢–ê–• - –≠–¢–û –ù–û–†...,1487,...,45450,1251.0,trello.com/b/IrHsvxUB/human-design-20,True,RU,True,False,False,30,59


## 1.2 Profile_file

In [397]:
Profile_file = train_file[['sid','profile_id','profile_name','firstname_lastname',
                         'following','followers','n_posts','url','is_business_account']]

In [398]:
Profile_file.isnull().sum()

sid                       0
profile_id                0
profile_name              0
firstname_lastname      381
following                 0
followers                 0
n_posts                   5
url                    7944
is_business_account       5
dtype: int64

### 1.2.1 following  and followers

we can find there are some thing wrong, we need drop them and reset values

In [399]:
temp = Profile_file[['following','followers']]
for i in range(0, len(temp)):
    try:
        temp.iloc[i] = pd.to_numeric(temp.iloc[i])
    except ValueError:
        print(temp.iloc[[i]])
        

                       following  followers
3344  2019-08-17 19:53:49.795000          0
                        following  followers
10143  2019-05-09 06:30:16.633000          0
                        following  followers
16910  2019-07-16 12:32:59.400000          1
                        following  followers
20912  2019-03-26 23:58:10.746000          0
                        following  followers
21354  2019-05-09 06:30:16.633000          0


### 1.2.2 n_post and is_busniess_aaccount

It is very lickly that tht when we delete those five special columns, those null values are also drop too 

In [400]:
Profile_file[Profile_file['n_posts'].isna()]

,sid,profile_id,profile_name,firstname_lastname,following,followers,n_posts,url,is_business_account
3344,43168879,1148067040,shipi0520,SHIPITAN(„Åó„Å¥„Åü„Çì),2019-08-17 19:53:49.795000,0,NaN,NaN,NaN
10143,9741848,45371780,afshanazad,Official Afshan,2019-05-09 06:30:16.633000,0,NaN,NaN,NaN
16910,40230055,1472338647,nibirnahid,Nibir Adnan,2019-07-16 12:32:59.400000,1,NaN,NaN,NaN
20912,3854721,333938620,royalex09,Alexüéìüíº‚è≥,2019-03-26 23:58:10.746000,0,NaN,NaN,NaN
21354,21058071,45371780,afshanazad,Official Afshan,2019-05-09 06:30:16.633000,0,NaN,NaN,NaN


check it any non numeric in the n_post and is_busniess_aaccount columns

In [401]:
Profile_file[~Profile_file[['n_posts','is_business_account']].applymap(np.isreal).all(1)]

,sid,profile_id,profile_name,firstname_lastname,following,followers,n_posts,url,is_business_account


According to the Profile_file, we can find:
 1. There are some special values in following and followers, just delete directly
 2. there are no special value int the n_post and is_busniess_aaccount
 3. There are lots of missing values in url, we can change it it have url, change into ture otherwise false

In [402]:
# delete the speical values in following and followers
train_file = train_file.drop([3344,10143,16910,20912,21354]).reset_index(drop=True)

# change url into true of false
train_file['url'] = np.where(train_file['url'].isnull(), False, True)

# delete profile_name  firstname_lastname 
train_file = train_file.drop(columns=['profile_name','firstname_lastname'])

In [403]:
train_file.shape

(21688, 22)

In [404]:
train_file.head(5)

,Unnamed: 0.1,Unnamed: 0,location_id,profile_id,sid,sid_profile,post_id,post_type,description,numbr_likes,...,followers,n_posts,url,is_business_account,cd,phone,aj_exact_city_match,aj_exact_country_match,lng,lat
0,11420,11755,340283947,829324748,28919418,295070,Bka_6xunYma,1,Cheers to wine O‚Äôclock ü•Ç (üì∏: @aob1085),4985,...,594895,3612.0,True,True,ES,True,False,False,2,41
1,24264,24967,2631322,199074048,42114489,4342099,BxaZS4fA_nV,1,#malcapuyaisland #bananaisland #bulogisland,57,...,428,95.0,False,False,PH,False,True,False,120,11
2,16286,16768,668571209,3280121227,19193237,2554203,BxmEzQUFocM,1,üî∏\\nSchwarzkehlchen (Saxicola rubicola) ‚Ä¢...,1576,...,36618,1259.0,True,True,DE,False,False,True,11,51
3,11139,11467,238461670,5625643374,11733129,3465973,Bjq5KFGlJvM,1,Eared Grebe spotted at one of my favourite spo...,29,...,126,39.0,False,False,CA,False,True,False,-118,52
4,27320,28111,1005642734,1557071010,8540244,3541402,BweviowATmj,1,–ü–õ–ê–ö–ê–¢–¨ –í –ú–û–°–¢–ê–• - –≠–¢–û –ù–û–†...,1487,...,45450,1251.0,True,True,RU,True,False,False,30,59


## 1.3 post_file

In [405]:
post_file = train_file[['sid','sid_profile','post_id','profile_id','location_id','post_type',
                      'description','numbr_likes','number_comments']]

In [406]:
post_file.isnull().sum()

sid                  0
sid_profile          0
post_id             31
profile_id           0
location_id          0
post_type            0
description        565
numbr_likes          0
number_comments      0
dtype: int64

#### 1.3.1 check special value in number_likes,numberr_comments and post_type

In [407]:
temp = post_file[['numbr_likes','number_comments']]
for i in range(0, len(temp)):
    try:
        temp.iloc[i] = pd.to_numeric(temp.iloc[i])
    except ValueError:
        print(temp.iloc[[i]])

In [408]:
set(post_file['post_type'])

{1, 2}

According to the post_file, we can find:
 1. number_likes, number_commnets and post_type have no speical values
 
Delete the description

In [409]:
# delete description
train_file = train_file.drop(columns=['description'])

## 1.4 delete all the id

In [410]:
# delete id
train_file = train_file.drop(columns=['sid','sid_profile','post_id','profile_id','location_id'])

In [411]:
train_file.shape

(21688, 16)

In [412]:
train_file.head(5)

,Unnamed: 0.1,Unnamed: 0,post_type,numbr_likes,number_comments,following,followers,n_posts,url,is_business_account,cd,phone,aj_exact_city_match,aj_exact_country_match,lng,lat
0,11420,11755,1,4985,78,420,594895,3612.0,True,True,ES,True,False,False,2,41
1,24264,24967,1,57,2,486,428,95.0,False,False,PH,False,True,False,120,11
2,16286,16768,1,1576,15,5378,36618,1259.0,True,True,DE,False,False,True,11,51
3,11139,11467,1,29,5,62,126,39.0,False,False,CA,False,True,False,-118,52
4,27320,28111,1,1487,11,544,45450,1251.0,True,True,RU,True,False,False,30,59


In [413]:
train_file.isnull().sum()

Unnamed: 0.1              0
Unnamed: 0                0
post_type                 0
numbr_likes               0
number_comments           0
following                 0
followers                 0
n_posts                   0
url                       0
is_business_account       0
cd                        0
phone                     0
aj_exact_city_match       0
aj_exact_country_match    0
lng                       0
lat                       0
dtype: int64